**Getting Started with Chatbot Arena Dataset**

This notebook provide starter code loading and inspecting the core dataset and the auxiliary datasets.


**Conversation Data**

The source dataset comes from https://huggingface.co/datasets/lmsys/chatbot_arena_conversations. The author describes the dataset as follows:

> This dataset contains 33K cleaned conversations with pairwise human preferences. It is collected from 13K unique IP addresses on the Chatbot Arena from April to June 2023. Each sample includes a question ID, two model names, their full conversation text in OpenAI API JSON format, the user vote, the anonymized user ID, the detected language tag, the OpenAI moderation API tag, the additional toxic tag, and the timestamp.

[Chatbot Arena](https://chat.lmsys.org/) is a platform where users can ask questions and two chatbots will provide answers. The user then votes on which chatbot provided the best answer. The dataset contains the questions, the answers, and the user votes.

You can find the processed dataset in `./chatbot-arena-conversations.jsonl.gz`. The dataset is in JSON line format and compressed using gzip. It has gone through the following preprocessing steps to make analysis easier:

- Removed non-English conversations.
- Removed conversations with more than one round.
- Removed conversations classified as toxic or harmful.

The dataset you will be working with contains `25322` rows (out of `33000` total rows) and `7` columns. The columns are:

- `question_id`: A unique identifier for the question.
- `model_a`: The name of the first chatbot model.
- `model_b`: The name of the second chatbot model.
- `winner`: The name of the chatbot model that won the user vote.
- `judge`: The anonymized user ID that voted.
- `conversation_a`: The conversation between the user and `model_a`.
- `conversation_b`: The conversation between the user and `model_b`.

An example of the data is as follows:

```json
{
  "question_id": "c0fc42c6f5f14f2aa5a89f71f8553730",
  "model_a": "vicuna-13b",
  "model_b": "koala-13b",
  "winner": "model_a",
  "judge": "arena_user_973",
  "conversation_a": [
    {
      "content": "Count from 1 to 10 with step = 3",
      "role": "user"
    },
    {
      "content": "1, 4, 7, 10\n\nCounting with a step of 3 means that you will increase the number by 3 each time. So, the first number is 1, the next is 4 (1 + 3), the next is 7 (4 + 3), and so on.",
      "role": "assistant"
    }
  ],
  "conversation_b": [
    {
      "content": "Count from 1 to 10 with step = 3",
      "role": "user"
    },
    {
      "content": "1, 4, 7, 10",
      "role": "assistant"
    }
  ]
}
```

Let's investigate the conversation data first. It is in JSON line format, compressed with `gzip`. You can load the data with `pd.read_json`.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

In [ ]:
df = pd.read_json(
    "/home/jovyan/shared/sp24_grad_project_data/nlp-arena-data/chatbot-arena-conversations.jsonl.gz",
    lines=True,
)
df.head(1)

Let's investigate the length distribution of the prompt. Do the arena users ask long or short questions? What are the average length of input for the chatbots?


In [ ]:
df["prompt"] = df["conversation_a"].str[0].str["content"]
df["prompt"].head(1)

In [ ]:
df["prompt_length"] = df["prompt"].str.len()
df["prompt_length"].describe()

Looks like the mean length of the prompt is about 200 characters, while the _median_ is 72 characters. This suggests that the distribution is right-skewed! Let's visualize this.


In [ ]:
# plot the distribution of the length of the prompt
plt.figure(figsize=(10, 6))
sns.histplot(df["prompt_length"], kde=True)

**Embedding Data**

Recall from homework 3, text embedding models transform natural language text into numerical vectors. The vector are generated in such a way that semantically similar text are close to each other in the vector space. In real world, these embeddings to find similar questions or to cluster the questions.

Concretely, `./chatbot-arena-prompts-embeddings.npy` contains the 256 dimensional text embeddings for each of the human question. The embeddings are generated using OpenAI's `text-embedding` model. We will explain what is embedding and how can you use them later in this notebook. It has shape (25322, 256) and have dtype('float32').

Let's try to compute the similarity using dot product of the first 1000 embeddings, and come up with examples of similar questions given the first question. This is am example of what you can do with embedding data. The ordering of the embeddings is the same as the ordering of the questions in the conversation data.


In [ ]:
embeddings = np.load(
    "/home/jovyan/shared/sp24_grad_project_data/nlp-arena-data/chatbot-arena-prompts-embeddings.npy"
)
embeddings.shape

In [ ]:
# Let's try to find the closest prompt to a given prompt

embeddings_sample = embeddings[:1000]

# compute the dot product between the embeddings
dot_product = np.dot(embeddings_sample, embeddings_sample.T)
dot_product.shape

Let's find the most similar questions to the a sample question. We can leverage the dot product results.


In [ ]:
source_prompt_idx = 23
source_prompt = df.iloc[source_prompt_idx].prompt
source_prompt

In [ ]:
top_k = 5
similar_promts_idx = np.argsort(dot_product[source_prompt_idx])[-top_k:][::-1]
similar_promts = df.iloc[similar_promts_idx].prompt
similar_promts.tolist()

**Topic Modeling and Hardness Score Data**

We used OpenAI's GPT 3.5 model to evaluate the difficulty of the questions. The model provides a hardness score from 1 to 10 for each question. The model also provides a topic modeling for each question. The topic modeling is a two-word description of the topic of the question. The topic modeling and hardness score can be used to understand the difficulty of the questions and to group similar questions together. Concretely:

- `./chatbot-arena-gpt3-scores.jsonl.gz` contains labels for the dataset you can use for later modeling tasks. It has the following fields:
  - `question_id`: The unique identifier for the question, as seen in `./chatbot-arena-conversations.jsonl.gz`.
  - `prompt`: The extracted human question. This is equivalent to the first message in `conversation_a` and `conversation_b` in `./chatbot-arena-conversations.jsonl.gz`.
  - `openai_scores_raw_choices_nested`: The response from OpenAI GPT 3.5 model (see later for the prompt). It contains the evaluated topic model, reason for a hardness score from 1 to 10, and the value. For each prompt, we have 3 responses from GPT 3.5 because it is a probablistic model. In fact, you will find in real world there are common multiple labels from different annotators for ground truth data. We extracted the fields into the following columns.
  - `topic_modeling_1`, `topic_modeling_2`, `topic_modeling_3`: The topic modeling for the first, second, and third response. Each topic should have two words.
  - `score_reason_1`, `score_reason_2`, `score_reason_3`: The reason for the hardness score for the first, second, and third response.
  - `score_value_1`, `score_value_2`, `score_value_3`: The hardness score for the first, second, and third response.

```json
{
  "question_id": "58210e39b3fd4441a2bd4a518bb44c2d",
  "prompt": "What is the difference between OpenCL and CUDA?",
  "openai_scores_raw_choices_nested": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\n    \"topic_modeling\": \"Technical Comparison\",\n    \"score_reason\": \"This prompt requires the AI to accurately compare and contrast two distinct technologies, OpenCL and CUDA. It assesses the AI's factual accuracy and knowledge of these technologies, as well as its ability to articulate the differences between them.\",\n    \"score_value\": 9\n}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    },
    {
      "finish_reason": "stop",
      "index": 1,
      "logprobs": null,
      "message": {
        "content": "{\n    \"topic_modeling\": \"Software Comparison\",\n    \"score_reason\": \"This prompt assesses the AI's factual accuracy in distinguishing between two similar but distinct software frameworks.\",\n    \"score_value\": 8\n}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    },
    {
      "finish_reason": "stop",
      "index": 2,
      "logprobs": null,
      "message": {
        "content": "{\n    \"topic_modeling\": \"Comparison, Technology\",\n    \"score_reason\": \"This prompt requires the AI to demonstrate knowledge of two different technologies, compare their features, and explain their distinctions. This task assesses the AI's factual accuracy and proficiency in understanding complex technological concepts.\",\n    \"score_value\": 9\n}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "topic_modeling_1": "Technical Comparison",
  "score_reason_1": "This prompt requires the AI to accurately compare and contrast two distinct technologies, OpenCL and CUDA. It assesses the AI's factual accuracy and knowledge of these technologies, as well as its ability to articulate the differences between them.",
  "score_value_1": 9,
  "topic_modeling_2": "Software Comparison",
  "score_reason_2": "This prompt assesses the AI's factual accuracy in distinguishing between two similar but distinct software frameworks.",
  "score_value_2": 8,
  "topic_modeling_3": "Comparison, Technology",
  "score_reason_3": "This prompt requires the AI to demonstrate knowledge of two different technologies, compare their features, and explain their distinctions. This task assesses the AI's factual accuracy and proficiency in understanding complex technological concepts.",
  "score_value_3": 9
}
```

_Warning_: This data can be messy! This is intentionally not cleaned up for you to resemble real world data. You are responsible to figuring out the irregularities and cleaning it up. The following cells demostrate the example of messy data.


In [ ]:
topic_and_hardness = pd.read_json(
    "/home/jovyan/shared/sp24_grad_project_data/nlp-arena-data/chatbot-arena-gpt3-scores.jsonl.gz",
    lines=True,
)
topic_and_hardness.head(1)

In [ ]:
# Example of messy data, there are other problems with the data but we are just demostrating one

topic_and_hardness["score_value_1"].isna().sum()

In [ ]:
# Example of messy data, there are other problems with the data but we are just demostrating one

topic_and_hardness["score_value_1"][topic_and_hardness["score_value_1"].apply(
    lambda x: isinstance(x, list))].head()